In [9]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
data = pd.read_csv('/kaggle/input/e-commerce/E-Commerce.csv')

# Display the first few rows of the dataset
print("First few rows of the dataset:")
print(data.head())

First few rows of the dataset:
   UserID  ProductID ProductCategory  Rating         Timestamp
0       1        101     Electronics       5  01-08-2024 10:00
1       1        102        Clothing       3  01-08-2024 11:30
2       2        103  Home & Kitchen       4  01-08-2024 12:00
3       2        104     Electronics       2  01-08-2024 13:00
4       3        105           Books       5  02-08-2024 09:30


In [10]:
# Create the user-item interaction matrix
interaction_matrix = data.pivot(index='UserID', columns='ProductID', values='Rating').fillna(0)

# Display the interaction matrix
print("\nUser-Item Interaction Matrix:")
print(interaction_matrix.head())

# Apply SVD (Singular Value Decomposition)
svd = TruncatedSVD(n_components=5)  # Number of components can be adjusted
matrix = svd.fit_transform(interaction_matrix)
svd_matrix = svd.components_

# Reconstruct the interaction matrix from the SVD components
predicted_ratings = np.dot(matrix, svd_matrix)

# Convert the predicted ratings back to a DataFrame
predicted_ratings_df = pd.DataFrame(predicted_ratings, index=interaction_matrix.index, columns=interaction_matrix.columns)

# Function to get recommendations for a specific user
def get_recommendations(user_id, num_recommendations=5):
    if user_id not in predicted_ratings_df.index:
        raise ValueError(f"User ID {user_id} not found in the data.")
    
    # Get the predicted ratings for the specified user
    user_predictions = predicted_ratings_df.loc[user_id]
    
    # Get the products the user has already rated
    rated_products = interaction_matrix.loc[user_id]
    
    # Filter out already rated products
    unrated_products = user_predictions[rated_products == 0]
    
    # Sort the unrated products by predicted rating in descending order
    recommendations = unrated_products.sort_values(ascending=False)
    
    return recommendations.head(num_recommendations)


User-Item Interaction Matrix:
ProductID  101  102  103  104  105  106  107  108  109  110  ...  116  117  \
UserID                                                       ...             
1          5.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  5.0  0.0   
2          0.0  0.0  4.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  3.0   
3          0.0  0.0  0.0  0.0  5.0  4.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
4          0.0  0.0  0.0  0.0  0.0  0.0  3.0  5.0  0.0  0.0  ...  0.0  0.0   
5          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  4.0  3.0  ...  0.0  0.0   

ProductID  118  119  120  121  122  123  124  125  
UserID                                             
1          0.0  0.0  0.0  4.0  0.0  0.0  0.0  0.0  
2          0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  
3          2.0  0.0  0.0  0.0  0.0  3.0  0.0  0.0  
4          0.0  4.0  0.0  0.0  0.0  0.0  5.0  0.0  
5          0.0  0.0  5.0  0.0  0.0  0.0  0.0  4.0  

[5 rows x 25 columns]


In [11]:
# Example usage
user_id = 1  # Change this to the user ID you want to get recommendations for
recommendations = get_recommendations(user_id, num_recommendations=5)
print(f"\nTop 5 recommendations for User {user_id}:")
print(recommendations)


Top 5 recommendations for User 1:
ProductID
114    1.827397e-15
108    1.827397e-15
124    1.827397e-15
105    9.766561e-16
107    9.063751e-16
Name: 1, dtype: float64
